## **Deployment**

Demo the predictive machine learning model using [Gradio](https://gradio.app/)

In [1]:
!pip install gradio --quiet

Load the saved model and the dataframe

In [2]:
import pickle 
loaded_model = pickle.load(open('k-nn_model.pkl','rb'))

In [3]:
import pandas as pd

dataset_url = 'ai4i2020.csv'
df = pd.read_csv(dataset_url)

# rename dataset columns
df.rename(columns = {'Air temperature [K]':'Air temperature',
                     'Process temperature [K]':'Process temperature',
                     'Rotational speed [rpm]':'Rotational speed',
                     'Torque [Nm]':'Torque',
                     'Tool wear [min]':'Tool wear'},
          inplace = True)

df['Power'] = df['Rotational speed'] * df['Torque']

In [4]:
df[df['Machine failure'] == 1]


,UDI,Product ID,Type,Air temperature,Process temperature,Rotational speed,Torque,Tool wear,Machine failure,TWF,HDF,PWF,OSF,RNF,Power
50,51,L47230,L,298.9,309.1,2861,4.6,143,1,0,0,1,0,0,13160.6
69,70,L47249,L,298.9,309.0,1410,65.7,191,1,0,0,1,1,0,92637.0
77,78,L47257,L,298.8,308.9,1455,41.3,208,1,1,0,0,0,0,60091.5
160,161,L47340,L,298.4,308.2,1282,60.7,216,1,0,0,0,1,0,77817.4
161,162,L47341,L,298.3,308.1,1412,52.3,218,1,0,0,0,1,0,73847.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9758,9759,L56938,L,298.6,309.8,2271,16.2,218,1,1,0,0,0,0,36790.2
9764,9765,L56944,L,298.5,309.5,1294,66.7,12,1,0,0,1,0,0,86309.8
9822,9823,L57002,L,298.5,309.4,1360,60.9,187,1,0,0,0,1,0,82824.0
9830,9831,L57010,L,298.3,309.3,1337,56.1,206,1,0,0,0,1,0,75005.7


In [5]:
import numpy as np

# normalize the feature using z-score normalization
def z_score(feature, value):
    mean = np.mean(df[feature])
    std = np.std(df[feature])
    return (float(value) - mean) / std

In [6]:
import gradio as gr
from sklearn.preprocessing import StandardScaler

def predict(air_temperature, process_temperature, rotational_speed, torque, tool_wear, type):
    # normalize the inputs
    air_temperature = z_score('Air temperature', air_temperature)
    process_temperature = z_score('Process temperature', process_temperature)
    rotational_speed = z_score('Rotational speed', rotational_speed)
    torque = z_score('Torque', torque)
    tool_wear = z_score('Tool wear', tool_wear)
    power = z_score('Power', torque * rotational_speed)

    mapping = {'L': 0, 'M': 1, 'H': 2}
    temp = [{'Type':mapping[type], 'Air temperature':air_temperature,
             'Process temperature':process_temperature, 'Rotational speed':rotational_speed,
             'Torque':torque, 'Tool wear':tool_wear, 'Power':power}]

    input_data = pd.DataFrame(temp) 
    prediction = loaded_model.predict_proba(input_data)

    classes = ["No failure", "Machine failure"]
    prediction_dict = dict
    for record in prediction:
        prediction_dict = {classes[i] : record[i] for i in range(2)}
    
    maintenance = "No action required"
    if max(prediction_dict, key=prediction_dict.get) != "No failure":
        maintenance = "Need maintenance"

    return prediction_dict, maintenance

# create the user interface (inputs and outputs)
demo = gr.Interface(
    fn=predict,
    inputs=[gr.Number(label="Air temperature"),
            gr.Number(label="Process temperature"),
            gr.Number(label="Rotational speed"),
            gr.Number(label="Torque"),
            gr.Number(label="Tool wear"),
            gr.Radio(["L", "M", "H"], label="Type")],
    outputs=[gr.Label(num_top_classes=2, label="Result"), gr.components.Textbox(label="Action")]
)
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
